In [14]:
# load libraries

import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
from sklearn.base import clone
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector as selector

import plotly.graph_objects as go
import plotly.express as px

from experiment_code import constants as const
#from experiment_code.preprocess import ExpSentences
from experiment_code.visualization.visualize import CoRTLanguageExp
from models import run_models
from models import modeling

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# initialize class 
#cort = CoRTLanguageExp()

# load df
#df = cort.load_dataframe()

os.chdir(const.Defaults.PROCESSED_DIR)
df = pd.read_csv("merged_preprocessed_dataframe.csv")
df.columns #change format (e.g.index=False)

Index(['Unnamed: 0', 'local_date', 'experiment_id', 'experiment_version',
       'participant_public_id', 'participant_id', 'task_name', 'task_version',
       'spreadsheet_version', 'spreadsheet_row', 'sentence_num', 'zone_type',
       'rt', 'response', 'attempt', 'correct', 'incorrect', 'display',
       'block_num', 'randomise_blocks', 'full_sentence', 'last_word',
       'sampled', 'CoRT_descript', 'CoRT_mean', 'condition_name', 'CoRT_std',
       'cloze_descript', 'cloze_probability', 'dataset', 'random_word',
       'target_word', 'word_count', 'group', 'cause_effect', 'dynamic_verb',
       'orientation', 'negative', 'tense', 'spelling_modified', 'trial_type',
       'version', 'version_descript', 'group_cloze_condition',
       'group_CoRT_condition', 'group_trial_type', 'cloze_cort', 'public_id',
       'gender', 'years_of_education', 'age', 'MOCA_mean', 'MOCA_std',
       'SARA_mean', 'SARA_std'],
      dtype='object')

In [16]:
# run models

# NOTE:
# model functions have been saved in modeling.py and the high-level model routine has been saved in run_models.py
# to modify the models, go to get_model_features in modelling.py and hardcode in new model features

fig, models = run_models.run(dataframe=df,
              model_names=['rt','cort', 'cloze','cort+cloze'])

#'cort', 'cloze','cort+cloze

fig
#getting stats from?

fitting rt model for c01
fitting rt model for c02
fitting rt model for c03
fitting rt model for c04
fitting rt model for c06
fitting rt model for c07
fitting rt model for c08
fitting rt model for c09
fitting rt model for c10
fitting rt model for c11
fitting rt model for c12
fitting rt model for c13
fitting rt model for c14
fitting rt model for c15
fitting rt model for c16
fitting rt model for c17
fitting rt model for c18
fitting rt model for p01
fitting rt model for p02
fitting rt model for p03
fitting rt model for p04
fitting rt model for p05
fitting rt model for p07
fitting rt model for p09
fitting rt model for p10
fitting rt model for p12
fitting rt model for p13
fitting rt model for p14
fitting rt model for p15
fitting rt model for p16
fitting rt model for p17
fitting cort model for c01
fitting cort model for c02
fitting cort model for c03
fitting cort model for c04
fitting cort model for c06
fitting cort model for c07
fitting cort model for c08
fitting cort model for c09
fitting c

In [7]:
models

,train_rmse,cv_rmse,model_name,subj
0,0.365458,0.457913,rt,c01
1,1.053191,1.312083,rt,c02
2,0.904520,1.119117,rt,c03
3,0.775192,0.944664,rt,c04
4,0.677251,0.834038,rt,c06
5,0.910974,1.121445,rt,c07
6,0.825606,1.017259,rt,c08
7,0.829013,1.008920,rt,c09
8,0.677761,0.841814,rt,c10
9,0.535976,0.651825,rt,c11


In [6]:
def get_model_features(model_name):
    """Hardcode model features for `model_name`
    
    Args: 
        model_name (str): 'eyetracking' etc
    Returns: 
        quant_features (list of str), qual_features (list of str)
    """
    #ALTER FOR EXP
    if model_name=='cort':
        quant_features = ['correct']
        qual_features = ['CoRT_descript', 'group', 'trial_type']
    elif model_name=='rt':
        quant_features = ['rt']
        qual_features = []
    elif model_name=='cloze':
        quant_features = ['correct']
        qual_features = ['cloze_descript', 'group', 'trial_type']
    elif model_name=='demog':
        quant_features = ['years_of_education', 'age', 'MOCA_total_score', 'SARA_total_score']
        qual_features = ['group', 'gender'] #input array of etiology? ignore skewed gender?
    elif model_name=='cort+cloze':
        quant_features = ['correct']
        qual_features = ['CoRT_descript', 'cloze_descript', 'group', 'trial_type']
    elif model_name=='combined':
        quant_features = ['correct','years_of_education', 'age', 'MOCA_total_score', 'SARA_total_score']
        qual_features = ['CoRT_descript', 'cloze_descript', 'group', 'trial_type']
    else:
        print('please define model type')
        
    return quant_features, qual_features

In [7]:
def model_fitting(dataframe, model_name, subj_id, data_to_predict):
    """Fits n models (n=length of model names) on `subj_id`
    
    Args: 
        dataframe (pd dataframe): containing columns `corr_resp` and model predictors
        model_names (str): model name as defined in `get_model_features`
        subj_id (str): we're fitting the model on individual subjects
        data_to_predict (str): `corr_resp` or `rt`
    Returns: 
        fitted model (sklearn obj)
    """
    
    # filter dataframe for `subj`
    dataframe = dataframe[dataframe['participant_id']==subj_id]

    # shuffle data before splitting
    shuffled_data = dataframe.sample(frac=1., random_state=42)

    # now split up the datasets into training and testing
    train, test = train_test_split(shuffled_data, test_size=0.1, random_state=83)

    # get model features
    quant_features, qual_features = get_model_features(model_name)

    # define model
    model = define_model(quant_features, qual_features)

    # fit model
    fitted_model = fit_model(model, X=train, y=train[data_to_predict])
    #inspect weights 

    print(f'fitting {model_name} model for {subj_id}')

    return fitted_model, train, test

In [30]:
#for model_name in model_names:

    #for subj in const.subj_id:

fitted_model, train, test = modeling.model_fitting(dataframe=df, 
                                        model_name='cort', 
                                        subj_id=const.subj_id[1],
                                        data_to_predict='rt')

fitting cort model for c02


In [29]:
test

,Unnamed: 0,local_date,experiment_id,experiment_version,participant_public_id,participant_id,task_name,task_version,spreadsheet_version,spreadsheet_row,...,group_trial_type,cloze_cort,public_id,gender,years_of_education,age,MOCA_mean,MOCA_std,SARA_mean,SARA_std
473,473,22/07/2020 13:54:35,21978.0,5.0,sAE,c02,cort_prepilot_task,34.0,FINAL sentences,312.0,...,control: meaningless,"CoRT, high cloze",sAE,M,16.0,51.40274,26.0,NaN,NaN,NaN
496,496,22/07/2020 13:56:57,21978.0,5.0,sAE,c02,cort_prepilot_task,34.0,FINAL sentences,335.0,...,control: meaningful,"non-CoRT, high cloze",sAE,M,16.0,51.40274,26.0,NaN,NaN,NaN
578,578,22/07/2020 14:06:43,21978.0,5.0,sAE,c02,cort_prepilot_task,34.0,FINAL sentences,29.0,...,control: meaningful,"non-CoRT, low cloze",sAE,M,16.0,51.40274,26.0,NaN,NaN,NaN
440,440,22/07/2020 13:49:55,21978.0,5.0,sAE,c02,cort_prepilot_task,34.0,FINAL sentences,278.0,...,control: meaningful,"CoRT, high cloze",sAE,M,16.0,51.40274,26.0,NaN,NaN,NaN
510,510,22/07/2020 13:58:20,21978.0,5.0,sAE,c02,cort_prepilot_task,34.0,FINAL sentences,349.0,...,control: meaningful,"CoRT, high cloze",sAE,M,16.0,51.40274,26.0,NaN,NaN,NaN
472,472,22/07/2020 13:54:29,21978.0,5.0,sAE,c02,cort_prepilot_task,34.0,FINAL sentences,311.0,...,control: meaningful,"CoRT, high cloze",sAE,M,16.0,51.40274,26.0,NaN,NaN,NaN
627,627,22/07/2020 14:11:50,21978.0,5.0,sAE,c02,cort_prepilot_task,34.0,FINAL sentences,78.0,...,control: meaningful,"CoRT, low cloze",sAE,M,16.0,51.40274,26.0,NaN,NaN,NaN
416,416,22/07/2020 13:47:28,21978.0,5.0,sAE,c02,cort_prepilot_task,34.0,FINAL sentences,254.0,...,control: meaningful,"CoRT, high cloze",sAE,M,16.0,51.40274,26.0,NaN,NaN,NaN
366,366,22/07/2020 13:41:26,21978.0,5.0,sAE,c02,cort_prepilot_task,34.0,FINAL sentences,138.0,...,control: meaningful,"CoRT, low cloze",sAE,M,16.0,51.40274,26.0,NaN,NaN,NaN
385,385,22/07/2020 13:44:08,21978.0,5.0,sAE,c02,cort_prepilot_task,34.0,FINAL sentences,223.0,...,control: meaningless,"CoRT, high cloze",sAE,M,16.0,51.40274,26.0,NaN,NaN,NaN


In [8]:
def rmse(y, yhat):
    """Calculates root mean square error
    
    Args: 
        y (np array): true y values
        yhat (np array): predicted y values
    Returns: 
        rmse (np array)
    """
    return np.sqrt(np.mean((y - yhat)**2))

In [9]:
def cross_validate_rmse(model, train, data_to_predict):
    """Cross validates training data
    
    Args: 
        model (sklearn obj): must contain `fit` and `predict`
        train (pd dataframe): shape (n_obs, regressors)
        data_to_predict (str): `corr_resp` or `rt`
    Returns: 
        CV rmse (scalar)
    """
    model = clone(model)
    five_fold = KFold(n_splits=5)
    rmse_values = []
    for tr_ind, va_ind in five_fold.split(train):
        model.fit(train.iloc[tr_ind,:], train[data_to_predict].iloc[tr_ind])
        rmse_values.append(rmse(train[data_to_predict].iloc[va_ind], model.predict(train.iloc[va_ind,:])))
    return np.mean(rmse_values)

In [10]:
def compute_train_cv_error(model, train, test, data_to_predict):
    # Compute the training error for each model
    training_rmse = rmse(train[data_to_predict], model.predict(train))

    # Compute the cross validation error for each model
    validation_rmse = cross_validate_rmse(model, train, data_to_predict)

    # Compute the test error for each model (don't do this!)
    # WE SHOULD BE TESTING THE WINNING MODEL HERE (WHICHEVER MODEL YIELDED LOWEST TRAIN AND CV ERROR)
    #     test_rmse = rmse(test[data_to_predict], model.predict(test))
    
    return training_rmse, validation_rmse

In [11]:
def compare_models(model_results):
    """Does model comparison and visualizes RMSE for training and validation
    
    Args: 
        model_results (pd dataframe): must contain 'train_rmse', 'cv_rmse', 'model_name'
        
    Returns: 
        Plot comparing model performance (training RMSE versus CV RMSE)
    """
    model_names = model_results['model_name'].unique() #LINE NOT WORKING
    fig = go.Figure(
        [go.Bar(x=model_names, y=model_results['train_rmse'], name='Training RMSE'),
        go.Bar(x=model_names, y=model_results['cv_rmse'], name='CV RMSE')]
        )
    fig.update_yaxes({'range': [0.4, 0.45]})

    return fig

In [12]:
def define_model(quant_features, qual_features):
    """Define model
        
    Args: 
        quant_features (list of str): corresponding to column names of training data
        qual_features (list of str): corresponding to column names of training data
    Returns: 
        model (sklearn obj)
    """
    # transform numeric data
    numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
    
    # transform categorical data
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, quant_features), # selector(dtype_exclude="category")
            ('cat', categorical_transformer, qual_features)
        ])
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ("Ridge", Ridge())
    ])
    
    return model

In [13]:
def fit_model(model, X, y):
    """Fit model
    
    Args: 
        model (sklearn obj):
        X (pd dataframe): X, shape (n_obs, n_predictors)
        y (series): y, shape (n_obs, 1)
    Returns:
        fitted sklearn model
    """
    return model.fit(X, y)

,Unnamed: 0,local_date,experiment_id,experiment_version,participant_public_id,participant_id,task_name,task_version,spreadsheet_version,spreadsheet_row,...,group_trial_type,cloze_cort,public_id,gender,years_of_education,age,MOCA_mean,MOCA_std,SARA_mean,SARA_std
0,0,20/07/2020 16:18:19,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,222.0,...,control: meaningless,"non-CoRT, high cloze",sAA,M,16.0,51.920548,28.0,NaN,NaN,NaN
1,1,20/07/2020 16:18:26,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,223.0,...,control: meaningless,"CoRT, high cloze",sAA,M,16.0,51.920548,28.0,NaN,NaN,NaN
2,2,20/07/2020 16:18:32,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,224.0,...,control: meaningful,"non-CoRT, low cloze",sAA,M,16.0,51.920548,28.0,NaN,NaN,NaN
3,3,20/07/2020 16:18:39,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,225.0,...,control: meaningful,"non-CoRT, low cloze",sAA,M,16.0,51.920548,28.0,NaN,NaN,NaN
4,4,20/07/2020 16:18:45,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,226.0,...,control: meaningful,"CoRT, low cloze",sAA,M,16.0,51.920548,28.0,NaN,NaN,NaN
5,5,20/07/2020 16:18:51,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,227.0,...,control: meaningless,"non-CoRT, low cloze",sAA,M,16.0,51.920548,28.0,NaN,NaN,NaN
6,6,20/07/2020 16:18:56,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,228.0,...,control: meaningful,"non-CoRT, high cloze",sAA,M,16.0,51.920548,28.0,NaN,NaN,NaN
7,7,20/07/2020 16:19:02,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,229.0,...,control: meaningless,"CoRT, low cloze",sAA,M,16.0,51.920548,28.0,NaN,NaN,NaN
8,8,20/07/2020 16:19:08,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,230.0,...,control: meaningless,"non-CoRT, low cloze",sAA,M,16.0,51.920548,28.0,NaN,NaN,NaN
9,9,20/07/2020 16:19:14,21978.0,5.0,sAA,c01,cort_prepilot_task,34.0,FINAL sentences,231.0,...,control: meaningless,"non-CoRT, high cloze",sAA,M,16.0,51.920548,28.0,NaN,NaN,NaN
